In [1]:
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
RAW_DATA_FILE = "./data/2. Dane puste przebiegi.xlsx"

SOURCE_FILE = "./data/dane_puste_przebiegi_cleaned.xlsx"

## 1. Rename columns

In [ ]:
df = pd.read_excel(RAW_DATA_FILE)
new_names = {
    "Nr pełny": "full_number",
    "klient": "client",
    "Ciągnik": "tractor",
    "Trasa": "route",
    "Zał. miasto": "load_city",
    "Zał. kod pocztowy": "load_postal_code",
    "Zał. kraj": "load_country",
    "Roz. miasto": "unload_city",
    "Roz. kod pocztowy": "unload_postal_code",
    "Roz. kraj": "unload_country",
    "Podjęcie": "pickup_planned",
    "Podjęcie rzeczywiste": "pickup_actual",
    "Dostarczenie": "delivery_planned",
    "Dostarczenie rzeczywiste": "delivery_actual",
    "Km puste wg. mapy": "empty_km_map",
    "Km ład. wg. mapy": "loaded_km_map",
    "Km wg. mapy suma": "total_km_map"
}

df = df.rename(columns=new_names)


In [3]:
df.head()

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,26692/2025,129.0,WGM9815L,(DE) Nürnberg - (AT) Wörgl - (DE) Nürnberg,Nürnberg,90475,DE,Nürnberg,90475,DE,2025-09-08 00:30:00,2025-09-08 00:46:00,2025-09-08 22:00:00,2025-09-08 22:15:00,0.000,553.013,553.013
1,27082/2025,105.0,PZ4S023,(PL) Jarosty - (CZ) Praha 5,Jarosty,97-310,PL,Praha 5,NaN,CZ,2025-09-08 01:10:00,2025-09-07 23:55:00,2025-09-09 03:15:00,2025-09-09 01:47:27,0.000,540.058,540.058
2,27028/2025,61.0,PZ4R993,(DE) Euskirchen - (FR) amiens,Euskirchen,53881,DE,amiens,80013,FR,2025-09-08 05:00:00,2025-09-08 16:32:00,2025-09-08 15:00:00,2025-09-09 11:16:00,0.000,409.626,409.626
3,27026/2025,47.0,WGM8283K,(CZ) Modřice - (CZ) Pelhřimov - (CZ) VRSKMAN,Modřice,66442,CZ,VRSKMAN,43111,CZ,2025-09-08 05:30:00,2025-09-08 05:48:28,2025-09-08 12:30:00,2025-09-08 14:04:21,318.425,351.447,669.872
4,27049/2025,105.0,WGM5118H,(PL) Chlastawa - (DE) Berlin,Chlastawa,66-210,PL,Berlin,13597,DE,2025-09-08 06:00:00,2025-09-09 00:34:00,2025-09-09 06:00:00,2025-09-09 05:17:00,0.789,216.089,216.878


## 2. Clean data

In [4]:
before = len(df)

In [ ]:
# 1️⃣ Remove commas, spaces, and quotes from the beginning and end
df["load_city"] = df["load_city"].str.strip(" ,\"'")
df["unload_city"] = df["unload_city"].str.strip(" ,\"'")

# 2️⃣ Remove NaN values before applying conversions
# df = df.dropna(subset=["load_city", "unload_city"])

# 3️⃣ Remove rows with empty or whitespace-only values
df = df[(df["load_city"].str.strip() != "") & (df["unload_city"].str.strip() != "")]

# 4️⃣ Normalize: lowercase + strip
df["load_city"] = df["load_city"].str.strip().str.lower()
df["unload_city"] = df["unload_city"].str.strip().str.lower()

# 5️⃣ Fix typos and special exceptions
df[["load_city", "unload_city"]] = df[["load_city", "unload_city"]].replace({
    "csomor": "csömör",
    "kolonia": "köln",
    "vieux- thann": "vieux-thann",
    "saint-quentin fallavier": "saint-quentin-fallavier",
    "saint quentin fallavier": "saint-quentin-fallavier",
    "st quentin fallavier": "saint-quentin-fallavier",
})

# df[["load_city", "unload_city"]] = df[["load_city", "unload_city"]].replace({
#     "csomor": "csömör",
#     "kolonia": "köln",
#     "vieux- thann": "vieux-thann",
#     "saint-quentin fallavier": "saint-quentin-fallavier",
#     "saint quentin fallavier": "saint-quentin-fallavier",
#     "st quentin fallavier": "saint-quentin-fallavier",
#     "rakovnik": "rakovník",
#     "vaihingen": "vaihingen an der enz",
#     "mockmuhl": "möckmühl",
#     "moeckmuehl": "möckmühl",
#     "buerstadt": "bürstadt",
#     "burstadt": "bürstadt",
#     "suelzetal": "sülzetal",
#     "sulzetal": "sülzetal",
#     "henstedt ulzburg": "henstedt-ulzburg",
#     "neumunster": "neumünster",
#     "moissy cramayel": "moissy-cramayel",
#     "trith-saint-leger": "trith-saint-léger",
#     "gyal": "gyál",
#     "gostyn": "gostyń",
# })


In [6]:
after = len(df)
print(f"Usunięto {before - after} wierszy z pustymi lub nullowymi miastami.")

Usunięto 0 wierszy z pustymi lub nullowymi miastami.


In [ ]:
df

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,26692/2025,129.0,WGM9815L,(DE) Nürnberg - (AT) Wörgl - (DE) Nürnberg,nürnberg,90475,DE,nürnberg,90475,DE,2025-09-08 00:30:00,2025-09-08 00:46:00,2025-09-08 22:00:00,2025-09-08 22:15:00,0.000,553.013,553.013
1,27082/2025,105.0,PZ4S023,(PL) Jarosty - (CZ) Praha 5,jarosty,97-310,PL,praha 5,NaN,CZ,2025-09-08 01:10:00,2025-09-07 23:55:00,2025-09-09 03:15:00,2025-09-09 01:47:27,0.000,540.058,540.058
2,27028/2025,61.0,PZ4R993,(DE) Euskirchen - (FR) amiens,euskirchen,53881,DE,amiens,80013,FR,2025-09-08 05:00:00,2025-09-08 16:32:00,2025-09-08 15:00:00,2025-09-09 11:16:00,0.000,409.626,409.626
3,27026/2025,47.0,WGM8283K,(CZ) Modřice - (CZ) Pelhřimov - (CZ) VRSKMAN,modřice,66442,CZ,vrskman,43111,CZ,2025-09-08 05:30:00,2025-09-08 05:48:28,2025-09-08 12:30:00,2025-09-08 14:04:21,318.425,351.447,669.872
4,27049/2025,105.0,WGM5118H,(PL) Chlastawa - (DE) Berlin,chlastawa,66-210,PL,berlin,13597,DE,2025-09-08 06:00:00,2025-09-09 00:34:00,2025-09-09 06:00:00,2025-09-09 05:17:00,0.789,216.089,216.878


In [ ]:
df.to_excel(SOURCE_FILE, index=False)

print(f"✅ Dane wyczyszczone i zapisane do: {SOURCE_FILE}")

✅ Dane wyczyszczone i zapisane do: ./data/dane_puste_przebiegi_cleaned.xlsx
